In [11]:
import os
import requests
from bs4 import BeautifulSoup
import re
import csv
import pandas as pd
from newsapi import NewsApiClient
from urllib.parse import urlparse, parse_qs
from openpyxl import load_workbook
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('vader_lexicon')
nltk.download('sentiwordnet')
nltk.download('averaged_perceptron_tagger')

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem import PorterStemmer
from nltk.corpus import wordnet as wn
from nltk.corpus import sentiwordnet as swn
from nltk.tag import pos_tag

from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.stem import WordNetLemmatizer

from readability.exceptions import ReadabilityException

import stanza
stanza.download('en')  # Download the English model

from readability import Readability

import spacy
nlp_spacy = spacy.load("en_core_web_sm")

import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
from gensim.corpora import Dictionary
from gensim.models.ldamodel import LdaModel

import pyLDAvis.gensim
import pyLDAvis.gensim_models as gensimvis
from gensim.models import Phrases
from gensim.models.phrases import Phraser

import statistics
import numpy as np
import pandas as pd
from IPython.display import display

import csv
from tabulate import tabulate
import newspaper

import torch
torch.cuda.empty_cache()
torch.cuda.memory_summary(device=None, abbreviated=False)

import os
import requests
from bs4 import BeautifulSoup
import re

from openpyxl import Workbook
import openpyxl

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)


[nltk_data] Downloading package punkt to /home/pierluigi/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/pierluigi/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /home/pierluigi/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /home/pierluigi/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package sentiwordnet to
[nltk_data]     /home/pierluigi/nltk_data...
[nltk_data]   Package sentiwordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/pierluigi/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


2023-06-08 10:34:34 INFO: Downloading default packages for language: en (English) ...
2023-06-08 10:34:35 INFO: File exists: /home/pierluigi/stanza_resources/en/default.zip
2023-06-08 10:34:42 INFO: Finished downloading models and saved to /home/pierluigi/stanza_resources.


In [12]:
nltk.download('stopwords')
stop_words = set(nltk.corpus.stopwords.words('english'))
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

nlp = spacy.load("en_core_web_sm")

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/pierluigi/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/pierluigi/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/pierluigi/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /home/pierluigi/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [13]:
def save_article_text(url):
    # Set headers to mimic a web browser
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
    }

    # Send a GET request to the URL with headers
    response = requests.get(url, headers=headers)
    response.raise_for_status()

    # Parse the HTML content using BeautifulSoup
    soup = BeautifulSoup(response.content, 'html.parser')

    # Extract the title, subtitle, description, and main text
    title_element = soup.find('title')
    title = title_element.text.strip() if title_element else ""

    subtitle_element = soup.find('meta', attrs={'name': 'description'})
    subtitle = subtitle_element['content'].strip() if subtitle_element and 'content' in subtitle_element.attrs else ""

    description_element = soup.find('meta', attrs={'name': 'og:description'})
    description = description_element['content'].strip() if description_element and 'content' in description_element.attrs else ""

    # Find and exclude unwanted elements by class names or content patterns
    unwanted_elements = soup.find_all(['script', 'style', 'a', 'div', 'span'], class_=['follow-us', 'newsletter', 'advertisement'])
    patterns_to_exclude = ['next article', 'read next', 'correlated']
    for element in unwanted_elements:
        if any(pattern in str(element).lower() for pattern in patterns_to_exclude):
            element.extract()

    # Find and exclude footer container and "All rights reserved" text
    footer_elements = soup.find_all(['footer', 'div'], class_=['footer', 'bottom-footer'])
    for element in footer_elements:
        element.extract()
    all_rights_reserved_elements = soup.find_all(text=re.compile(r'\bAll rights reserved\b', re.IGNORECASE))
    for element in all_rights_reserved_elements:
        element.extract()

    # Find the main text element(s) based on the HTML structure of the page
    main_text_elements = soup.find_all('p')
    main_text = "\n\n".join([element.text.strip() for element in main_text_elements if element.text.strip()])

    # Set the subtitle to the description if it is empty
    if not subtitle:
        subtitle = description.strip()

    # Concatenate the extracted strings
    article_text = f"{title}\n\n{subtitle}\n\n{main_text}"

    return article_text

In [14]:
def extract_real_url(google_news_url):
    response = requests.get(google_news_url, cookies = {'CONSENT' : 'YES+'})
    real_url = response.url
    return real_url

In [15]:
def process_data(data_dict):
    processed_data = {}
    
    for sheet_name, data_list in data_dict.items():
        sheet_data = {}
        
        for index, url in enumerate(data_list, start=1):
            try:
                article_url = extract_real_url(url)
                article_text = save_article_text(article_url)
                key = f"article{index}"
                sheet_data[key] = article_text
            except (requests.exceptions.RequestException, requests.exceptions.HTTPError) as e:
                # Handle the exception and continue with the next URL
                print(f"Error processing URL: {url}")
                print(f"Error message: {str(e)}")
                continue
        
        processed_data[sheet_name] = sheet_data
    
    return processed_data

In [16]:
def preprocess_text(text):
    # Tokenize the text
    tokens = nltk.word_tokenize(text)
    
    # Perform lemmatization
    lemmatizer = WordNetLemmatizer()
    lemmatized_words = [lemmatizer.lemmatize(token) for token in tokens]
    
    # Generate bigrams
    bigrams = list(nltk.bigrams(lemmatized_words))
    lemmatized_bigrams = [bigram for bigram in bigrams]
    
    return lemmatized_bigrams

def apply_lda_to_articles(data_dict, min_topics, max_topics):
    lda_results = {}  # Dictionary to store the best number of topics for each sheet
    
    for sheet_name, sheet_data in data_dict.items():
        sheet_results = []
        
        for article_key, article_text in sheet_data.items():
            lemmatized_bigrams = preprocess_text(article_text)
            
            # Create Dictionary
            id2word = corpora.Dictionary(lemmatized_bigrams)
            
            # Create Corpus
            texts = lemmatized_bigrams
            corpus = [id2word.doc2bow(text) for text in texts]
            
            if not corpus:
                print("Error: No data in the corpus. Unable to compute LDA model for article:", article_key)
                continue
            
            best_num_topics = None
            best_coherence = float('-inf')
            
            for num_topics in range(min_topics, max_topics + 1):
                lda_model = gensim.models.LdaModel(corpus=corpus,
                                                   id2word=id2word,
                                                   num_topics=num_topics,
                                                   random_state=100,
                                                   update_every=1,
                                                   chunksize=100,
                                                   passes=10,
                                                   alpha='auto',
                                                   per_word_topics=True)
                coherence_model_lda = CoherenceModel(model=lda_model, texts=lemmatized_bigrams, dictionary=id2word, coherence='c_v')
                coherence = coherence_model_lda.get_coherence()
                
                if coherence > best_coherence:
                    best_num_topics = {'num_topics': num_topics}
                    best_coherence = coherence
            
            if best_num_topics is not None:
                lda_results[(sheet_name, article_key)] = best_num_topics
            else:
                print("Error: Unable to determine the optimal number of topics for article:", article_key)
        
        lda_results[sheet_name] = sheet_results
    
    return lda_results


In [17]:
def save_scores_to_excel_lda(processed_data_list, lda_scores_list, output_file_path):
    output_workbook = openpyxl.Workbook()
    processed_articles = 0
    
    for i, processed_data in enumerate(processed_data_list):
        lda_scores = lda_scores_list[i]
        sheet_name = f"Sheet {i+1}"
        
        if i == 0:
            output_sheet = output_workbook.active
            output_sheet.title = sheet_name
        else:
            output_sheet = output_workbook.create_sheet(title=sheet_name)
        
        output_sheet['A1'] = 'Date'
        
        row = 2
        
        for sheet_name, sheet_data in processed_data.items():
            output_sheet.cell(row=row, column=1, value=sheet_name)
            column = 2
            
            for article_index, (article_key, article_text) in enumerate(sheet_data.items()):
                if (sheet_name, article_key) in lda_scores:
                    scores = lda_scores[(sheet_name, article_key)]
                    output_sheet.cell(row=1, column=column, value=f"LDA Number of topics {article_index+1}")
                    output_sheet.cell(row=row, column=column, value=scores['num_topics'])
                else:
                    output_sheet.cell(row=1, column=column, value=f"LDA Number of topics {article_index+1}")
                    output_sheet.cell(row=row, column=column, value='N/A')
                
                column += 1
            row += 1
            processed_articles += 1
            print(f"Sheet: {sheet_name} | Article: {article_key} done.")

    
    output_workbook.save(output_file_path)


In [18]:
def excel_to_csv(excel_file):
    # Load the Excel file
    xls = pd.ExcelFile(excel_file)

    # Get the sheet names
    sheet_names = xls.sheet_names

    # Iterate over each sheet and convert to CSV
    for sheet_name in sheet_names:
        # Read the sheet as a DataFrame
        df = pd.read_excel(excel_file, sheet_name=sheet_name)

        # Generate the CSV file name
        csv_file = f"{sheet_name}.csv"

        # Save the DataFrame as a CSV file
        df.to_csv(csv_file, index=False)

        print(f"CSV file '{csv_file}' created for sheet '{sheet_name}'")

In [19]:
def main():

    workbook = load_workbook('/home/pierluigi/Documents/echo_chambers_intership/Code analysis/NLP/Analysis/Text analytics/Text Analytics.xlsx')

    data_U1 = {}  # Dictionary to store data from range 1 - U1
    data_U2 = {}  # Dictionary to store data from range 2 - U2
    data_U3 = {}  # Dictionary to store data from range 3 - U3
    data_U4 = {}  # Dictionary to store data from range 4 - U4

    # Iterate over each sheet in the workbook
    for sheet_name in workbook.sheetnames:
        worksheet = workbook[sheet_name]

        # Specify the range of cells for data_U1
        range1_start_cell = 'C2'  # Replace with the starting cell of range 1
        range1_end_cell = 'L2'  # Replace with the ending cell of range 1
        
        data_list1 = []
        
        # Iterate over the cells within range 1
        for row in worksheet[range1_start_cell:range1_end_cell]:
            for cell in row:
                data_list1.append(cell.value)
        
        data_U1[sheet_name] = data_list1

        # Specify the range of cells for data_U2
        range2_start_cell = 'C3'  # Replace with the starting cell of range 2
        range2_end_cell = 'L3'  # Replace with the ending cell of range 2
        
        data_list2 = []
        
        # Iterate over the cells within range 2
        for row in worksheet[range2_start_cell:range2_end_cell]:
            for cell in row:
                data_list2.append(cell.value)
        
        data_U2[sheet_name] = data_list2

        # Specify the range of cells for data_U3
        range3_start_cell = 'C4'  # Replace with the starting cell of range 3
        range3_end_cell = 'L4'  # Replace with the ending cell of range 3
        
        data_list3 = []
        
        # Iterate over the cells within range 3
        for row in worksheet[range3_start_cell:range3_end_cell]:
            for cell in row:
                data_list3.append(cell.value)
        
        data_U3[sheet_name] = data_list3

        # Specify the range of cells for data_U4
        range4_start_cell = 'C5'  # Replace with the starting cell of range 4
        range4_end_cell = 'L5'  # Replace with the ending cell of range 4
        
        data_list4 = []
        
        # Iterate over the cells within range 4
        for row in worksheet[range4_start_cell:range4_end_cell]:
            for cell in row:
                data_list4.append(cell.value)
        
        data_U4[sheet_name] = data_list4

    workbook.close()

    # Process the data
    processed_data1 = process_data(data_U1)
    processed_data2 = process_data(data_U2)
    processed_data3 = process_data(data_U3)
    processed_data4 = process_data(data_U4)

    lda_scores1 = apply_lda_to_articles(processed_data1, 1, 10)
    lda_scores2 = apply_lda_to_articles(processed_data2, 1, 10)
    lda_scores3 = apply_lda_to_articles(processed_data3, 1, 10)
    lda_scores4 = apply_lda_to_articles(processed_data4, 1, 10)

    processed_data_list = [processed_data1, processed_data2, processed_data3, processed_data4]
    lda_scores_list = [lda_scores1, lda_scores2, lda_scores3, lda_scores4]

    output_file_path = '/home/pierluigi/Desktop/lda.xlsx'

    save_scores_to_excel_lda(processed_data_list, lda_scores_list, output_file_path)    

In [20]:
# Run the main function
if __name__ == "__main__":
    main()

Error processing URL: https://news.google.com/articles/CBMiSGh0dHBzOi8vd3d3LmxpZmVzdHlsZWFzaWEuY29tL2tsL3N0eWxlL2Zhc2hpb24vbWV0LWdhbGEtMjAyMy1iZXN0LW1lbWVzL9IBTGh0dHBzOi8vd3d3LmxpZmVzdHlsZWFzaWEuY29tL2tsL3N0eWxlL2Zhc2hpb24vbWV0LWdhbGEtMjAyMy1iZXN0LW1lbWVzL2FtcC8?hl=en-US&gl=US&ceid=US%3Aen
Error message: 403 Client Error: Forbidden for url: https://www.lifestyleasia.com/kl/style/fashion/met-gala-2023-best-memes/
Error processing URL: https://news.google.com/articles/CBMiUmh0dHBzOi8vc2NyZWVucmFudC5jb20vdmFuZGVycHVtcC1ydWxlcy10b20tc2FuZG92YWwtZGlzZGFpbi1rYXRpZS1kZWVwZXItbWVhbmluZy_SAQA?hl=en-US&gl=US&ceid=US%3Aen
Error message: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))
Error processing URL: https://news.google.com/articles/CBMiUWh0dHBzOi8vbmV3cy55YWhvby5jb20vc2VuYXRlLXJlcHVibGljYW5zLWluY2x1ZGluZy1tY2Nvbm5lbGwtZGVidC0yMjAwMzg1ODYuaHRtbNIBWWh0dHBzOi8vbmV3cy55YWhvby5jb20vYW1waHRtbC9zZW5hdGUtcmVwdWJsaWNhbnMtaW5jbHVkaW5nLW1jY29ubmVsbC1kZWJ0LT

In [21]:
def excel_to_csv(excel_file):
    # Load the Excel file
    xls = pd.ExcelFile(excel_file)

    # Get the sheet names
    sheet_names = xls.sheet_names

    # Iterate over each sheet and convert to CSV
    for sheet_name in sheet_names:
        # Read the sheet as a DataFrame
        df = pd.read_excel(excel_file, sheet_name=sheet_name)

        # Generate the CSV file name
        csv_file = f"{sheet_name}.csv"

        # Save the DataFrame as a CSV file
        df.to_csv(csv_file, index=False)

        print(f"CSV file '{csv_file}' created for sheet '{sheet_name}'")

In [22]:
excel_to_csv("/home/pierluigi/Desktop/lda.xlsx")

CSV file 'Sheet 1.csv' created for sheet 'Sheet 1'
CSV file 'Sheet 2.csv' created for sheet 'Sheet 2'
CSV file 'Sheet 3.csv' created for sheet 'Sheet 3'
CSV file 'Sheet 4.csv' created for sheet 'Sheet 4'
